# Demand and Travel Decision

In this tutorial we introduce the `demand` and `travel_decision` module of `mnms`. The management of the demand is made through the `DemandManager` classes. We cover two of them:

* `BaseDemandManager`
* `CSVDemandManager`
 
And the choice of the `User` path is made with the `travel_decision` module. We introduce two classes for that:

* `SimpleDecisionModel`
* `LogitDecisionModel`

## Generate a simple graph

We use a simple grid `flow_graph` generated with the function `create_grid_graph`, then we add one mobility service that cover all the `flow_graph`.

In [ ]:
import matplotlib.pyplot as plt

from mnms.graph.generation import create_grid_graph
from mnms.tools.render import draw_multimodal_graph 
from mnms.mobility_service.base import BaseMobilityService

mmgraph = create_grid_graph(2, 2)

car = BaseMobilityService('car', 10)

for node in mmgraph.flow_graph.nodes:
    car.add_node(f'C{node}', node)

for (up_node, down_node), link in mmgraph.flow_graph.links.items():
    car.add_link(f'C{up_node}_C{down_node}', 
                 f'C{up_node}', 
                 f'C{down_node}', 
                 costs={'time':1/10}, 
                 reference_links=[link.id])

mmgraph.add_mobility_service(car)

fig, ax = plt.subplots(figsize=(16, 9))
draw_multimodal_graph(ax, mmgraph, linkwidth=2, nodesize=5)

## Demand

### Random demand

In the following we create a random demand, the function `create_random_demand` return a `BaseDemandManager`.

In [ ]:
from mnms.demand.generation import create_random_demand

demand = create_random_demand(mmgraph, 
                              tstart="10:00:00", 
                              tend="12:00:00", 
                              cost_path='time', 
                              min_cost=0.4)
demand.show_users()

### CSV Predefined demand

We can use a predefined demand, the `CSVDemandManager` can read a sorted demand by time from a CSV file.

There is an example of demand in CSV format:

In [ ]:
import pandas as pd

pd.read_csv('data/simple_demand.csv', sep=';')

And how to load it in `mnms`:

In [ ]:
from mnms.demand import CSVDemandManager
from mnms.tools.time import Time

demand = CSVDemandManager('data/simple_demand.csv')

users = demand.get_next_departures(Time("07:00:00"), Time("07:30:00"))
users

## Travel Decision

The `travel_decision` module is used to choose the path of a a `User`, from a pair origin/destination it computes a path that will be later use in a `FlowMotor`.

### SimpleDecisionModel

The `SimpleDecisionModel` only compute the best shortest path for the `User` and set its `path` attribute

In [ ]:
from mnms.travel_decision import SimpleDecisionModel

travel_decision = SimpleDecisionModel(mmgraph)

first_user = users[0]
print(f"{first_user}, path: {first_user.path}")

In [ ]:
travel_decision(first_user)

print(f"{first_user}, path: {first_user.path}")

### LogitDecisionModel

The `LogitDecisionModel` compute `n` best shortest path and set a probability of choosing a path following this model:

\begin{equation}
p_j=\frac{e^{-\theta c_j}}{\sum_{i=1}^ne^{-\theta c_i}}
\end{equation}

Where:

* c is the cost of the path
* $\theta$ is the logit parameter
* n is the number of path

In [ ]:
from mnms.travel_decision import LogitDecisionModel

travel_decision = LogitDecisionModel(mmgraph,
                                     theta=0.01,
                                     n_shortest_path=2)

second_user = users[1]
print(f"{second_user}, path: {second_user.path}")

In [ ]:
travel_decision(second_user)

print(f"{second_user}, path: {second_user.path}")

### Rendering path

In [ ]:
from mnms.tools.render import draw_path, draw_flow_graph

fig, ax = plt.subplots(figsize=(16, 9))
draw_flow_graph(ax, mmgraph.flow_graph, linkwidth=2, nodesize=5)
draw_path(ax, mmgraph, second_user.path)